<a href="https://colab.research.google.com/github/safa-abidi/NLP-sentimentAnalysis-AraBert/blob/main/naive_bayes_more_unlabeled_HARD_dataset_30400_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd

import plotly.express as px

import xml.etree.ElementTree as ET
import pandas as pd
import os

from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
from typing import Any, Dict, List, Callable, Optional, Tuple, Union
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import torch

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.7 MB/s eta 0:00:00


In [4]:

from transformers import BertModel, BertTokenizer
import torch
import transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


# Dataset: labelled and unlabled

In [6]:
DATASETS_PATH = '/content/drive/MyDrive/'

%cd /content/drive/MyDrive/datasets

# Labeled Dataset
labeled_dataset_path = 'dataset_arabic_translated/Merged_Labelled_Arabic_fake_news_Dataset.xlsx'
labeled_dataset = pd.read_excel(labeled_dataset_path)
print("labeled dataset", labeled_dataset.shape)

###################################
# unlabelled dataset (HARD balanced)
unlabeled_dataset_path = 'balanced-reviews.txt'
unlabeled_data = pd.read_csv(unlabeled_dataset_path, delimiter='\t', header=None, names=['no', 'Hotel name', 'rating', 'user type', 'room type', 'nights', 'Review'], encoding='UTF-16')
print("unlabeled data", unlabeled_data.shape)
#print(unlabeled_data)
###################################
unlabeled_dataset = unlabeled_data.sample(n=30400)
print(unlabeled_dataset.shape)


/content/drive/MyDrive/datasets
labeled dataset (1600, 6)
unlabeled data (105699, 7)
(30400, 7)


In [7]:
# data split labeled_training and test
df_train_labeled, df_test = train_test_split(labeled_dataset, train_size=0.7, random_state=0)
print('Size of train dataframe: ', df_train_labeled.shape[0])
print('Size of test dataframe: ', df_test.shape[0])



Size of train dataframe:  1120
Size of test dataframe:  480


In [8]:
df_train = pd.concat([df_train_labeled, unlabeled_dataset], axis=0)
#print(df_train)
# Reset the index of the concatenated DataFrame
df_train = df_train.reset_index(drop=True)

## Encode the dataset labels
df_test['Label'] =  df_test.apply(lambda x: 0 if x['Class'] == 'خادع' else 1 if x['Class'] == 'صادقة' else -1, axis=1)
df_train['Label'] = df_train.apply(lambda x: 0 if x['Class'] == 'خادع' else 1 if x['Class'] == 'صادقة' else -1, axis=1)

print(df_train.shape)
print(df_test.shape)

(31520, 12)
(480, 7)


# BERT transformer

In [9]:


class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        tokenizer,
        model,
        max_length: int = 60,
        embedding_func: Optional[Callable[[torch.Tensor], torch.Tensor]] = None,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.Tensor, torch.Tensor]:
        # Tokenize the text with the provided tokenizer
        tokenized_text = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_length,truncation=True
        )["input_ids"]
        
        # padding
        padded_text = tokenized_text + [0]*(self.max_length-len(tokenized_text))

        # Create an attention mask telling BERT to use all words and ignore padded values
        attention_mask = np.where(np.array(padded_text) != 0, 1, 0)

        # bert takes in a batch so we need to unsqueeze the rows
        return (
            torch.tensor(padded_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.Tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        """No fitting necessary so we just return ourselves"""
        return self
    
def calculate_classification_metrics(
    y_true: np.array,
    y_pred: np.array,
    average: Optional[str] = None,
    return_df: bool = True,
) -> Union[Dict[str, float], pd.DataFrame]:
    """Computes f1, precision, recall, precision, kappa, accuracy, and support

    Args:
        y_true: The true labels
        y_pred: The predicted labels
        average: How to average multiclass results

    Returns:
        Either a dataframe of the performance metrics or a single dictionary
    """
    labels = unique_labels(y_true, y_pred)
    # get results
    precision, recall, f_score, support = sk_metrics.precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=average
    )

    kappa = sk_metrics.cohen_kappa_score(y_true, y_pred, labels=labels)
    accuracy = sk_metrics.accuracy_score(y_true, y_pred)

    # create a pandas DataFrame
    if return_df:
        results = pd.DataFrame(
            {
                "class": labels,
                "f_score": f_score,
                "precision": precision,
                "recall": recall,
                "support": support,
                "kappa": kappa,
                "accuracy": accuracy,
            }
        )
    else:
        results = {
            "f1": f_score,
            "precision": precision,
            "recall": recall,
            "kappa": kappa,
            "accuracy": accuracy,
        }

    return results

## AraBERT embedding

In [10]:
tokenizer = BertTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenized_dict = tokenizer.encode_plus(
    df_train["Review"].tolist(),
    add_special_tokens=True,
    max_length=512,  # Set maximum sequence length here
    truncation=True,
    )

bert_model = BertModel.from_pretrained("aubmindlab/bert-base-arabertv2")
tokenized_text = torch.tensor(tokenized_dict["input_ids"])
with torch.no_grad():
  embeddings = bert_model(torch.tensor(tokenized_text.unsqueeze(0)))

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-10-37db532cadec>:12: UserWarning: To copy construct from a tensor, it is recommended to use sou

# Model classifier

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import accuracy_score

In [12]:
X_train=df_train['Review']
y_train=df_train['Label'].values

X_test=df_test['Review']
y_test=df_test['Label'].values



bert_transformer = BertTransformer(tokenizer, bert_model)
## Logistic Regression model
model_nb = GaussianNB()
# Self training classifier
classifier = SelfTrainingClassifier(base_estimator=model_nb,
                                      threshold=0.9,
                                      criterion='threshold',
                                      max_iter=100, 
                                      verbose=True,)

model = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)

clf_ST = model.fit(X_train, y_train)

End of iteration 1, added 30158 new labels.
End of iteration 2, added 240 new labels.
End of iteration 3, added 2 new labels.


In [13]:
print('')
print('---------- Self Training Model - Summary ----------')

print('Classes: ', clf_ST.classes_)

print('---------- Self Training Model - Evaluation on Test Data ----------')
accuracy_score_ST = clf_ST.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_ST)
# Look at classification report to evaluate the model
print(classification_report(y_test, clf_ST.predict(X_test)))


---------- Self Training Model - Summary ----------
Classes:  [0 1]
---------- Self Training Model - Evaluation on Test Data ----------
Accuracy Score:  0.5666666666666667
              precision    recall  f1-score   support

           0       0.54      0.88      0.67       239
           1       0.69      0.25      0.37       241

    accuracy                           0.57       480
   macro avg       0.61      0.57      0.52       480
weighted avg       0.61      0.57      0.52       480

